# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

# Extract Data

In [1]:
import pandas as pd
import numpy as np
import math
import json

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio = portfolio.rename(columns={'id':'offerid'})

portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   offerid     10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   offerid     10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [3]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ K

In [4]:
profile['income'] = profile['income'].fillna(-1)
profile['gender'] = profile['gender'].fillna('unkown')

In [5]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [6]:
## merge dataframe

In [7]:
transcript = transcript.merge(profile,left_on='person',right_on='id',how='left')

In [8]:
event_dict ={
    'offer received':0
    , 'offer viewed':1
    , 'transaction':2
    , 'offer completed':3
}
transcript['event_order']=transcript['event'].apply(lambda x:event_dict[x])

transcript=transcript.sort_values(by=['person','time','event_order'])
transcript.head()

,person,event,value,time,gender,age,id,became_member_on,income,event_order
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,1
89291,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},228,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,2
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,1


,person,event,value,time,gender,age,id,became_member_on,income,event_order
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,1
89291,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},228,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,2
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,1


In [9]:
value_dict = transcript.value.to_dict()
for idx in transcript.index:
    transcript.at[idx,'keys'] = [x for x in value_dict[idx].keys()][0]
    transcript.at[idx,'values']  = [x for x in value_dict[idx].values()][0]

transcript=transcript.sort_values(by=['person','time','event_order','values'])

for idx in transcript.index:
    if transcript.at[idx,'keys'] !='amount':
        transcript.at[idx,'offer'] = transcript.at[idx,'values']

In [10]:
transcript = transcript.merge(portfolio,left_on='offer',right_on='offerid',how='left')

In [27]:
transcript.head()

,person,event,value,time,gender,age,id,became_member_on,income,event_order,...,values,offer,reward,channels,difficulty,duration,offer_type,offerid,startofduration,endofduration
0,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,5a8bc65990b245e5a138643cd4eb9837,5a8bc65990b245e5a138643cd4eb9837,0.0,"[email, mobile, social]",0.0,3.0,informational,5a8bc65990b245e5a138643cd4eb9837,168.0,240.0
3,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,0.0,"[web, email, mobile]",0.0,4.0,informational,3f207df678b143eea3cee63160fa8bed,336.0,432.0
5,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,408.0,528.0
9,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,fafdcd668e3743c1bb461111dcafc2a4,fafdcd668e3743c1bb461111dcafc2a4,2.0,"[web, email, mobile, social]",10.0,10.0,discount,fafdcd668e3743c1bb461111dcafc2a4,504.0,744.0
14,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,2906b810c7d4411798c6938adc9daaa5,2906b810c7d4411798c6938adc9daaa5,2.0,"[web, email, mobile]",10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,576.0,744.0


,person,event,value,time,gender,age,id,became_member_on,income,event_order,...,values,offer,reward,channels,difficulty,duration,offer_type,offerid,startofduration,endofduration
0,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,5a8bc65990b245e5a138643cd4eb9837,5a8bc65990b245e5a138643cd4eb9837,0.0,"[email, mobile, social]",0.0,3.0,informational,5a8bc65990b245e5a138643cd4eb9837,168.0,240.0
3,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,3f207df678b143eea3cee63160fa8bed,3f207df678b143eea3cee63160fa8bed,0.0,"[web, email, mobile]",0.0,4.0,informational,3f207df678b143eea3cee63160fa8bed,336.0,432.0
5,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,408.0,528.0
9,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,fafdcd668e3743c1bb461111dcafc2a4,fafdcd668e3743c1bb461111dcafc2a4,2.0,"[web, email, mobile, social]",10.0,10.0,discount,fafdcd668e3743c1bb461111dcafc2a4,504.0,744.0
14,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,0,...,2906b810c7d4411798c6938adc9daaa5,2906b810c7d4411798c6938adc9daaa5,2.0,"[web, email, mobile]",10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,576.0,744.0


## duration

In [11]:
# add start time and end time for each event 
for idx in transcript.index:
    if transcript.at[idx,'event']=='offer received':
        transcript.at[idx,'startofduration'] = transcript.at[idx,'time']
        transcript.at[idx,'endofduration'] = transcript.at[idx,'time'] + transcript.at[idx,'duration'] *24

    if transcript.at[idx,'event']=='offer completed':
        transcript.at[idx,'startofduration'] = transcript.at[idx,'time']- transcript.at[idx,'duration'] *24
        transcript.at[idx,'endofduration'] = transcript.at[idx,'time'] 
        


In [31]:
transcript = transcript.sort_values(by=['person','time','startofduration','event_order'])

person = transcript['person'].unique()


In [35]:
transcript[transcript['person']==person[1]]


,person,event,value,time,gender,age,id,became_member_on,income,event_order,...,values,offer,reward,channels,difficulty,duration,offer_type,offerid,startofduration,endofduration
20,00116118485d4dfda04fdbaba9a87b5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},168,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,168.0,288.0
21,00116118485d4dfda04fdbaba9a87b5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},216,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,1,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
22,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 0.7000000000000001},294,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 0.2},456,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 3.19},474,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,00116118485d4dfda04fdbaba9a87b5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},576,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,576.0,696.0
26,00116118485d4dfda04fdbaba9a87b5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},630,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,1,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN


,person,event,value,time,gender,age,id,became_member_on,income,event_order,...,values,offer,reward,channels,difficulty,duration,offer_type,offerid,startofduration,endofduration
20,00116118485d4dfda04fdbaba9a87b5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},168,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,168.0,288.0
21,00116118485d4dfda04fdbaba9a87b5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},216,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,1,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
22,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 0.7000000000000001},294,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 0.2},456,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,00116118485d4dfda04fdbaba9a87b5c,transaction,{'amount': 3.19},474,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,2,...,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,00116118485d4dfda04fdbaba9a87b5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},576,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,0,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,576.0,696.0
26,00116118485d4dfda04fdbaba9a87b5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},630,unkown,118,00116118485d4dfda04fdbaba9a87b5c,20180425,-1.0,1,...,f19421c1d4aa40978ebb69ca19b0e20d,f19421c1d4aa40978ebb69ca19b0e20d,5.0,"[web, email, mobile, social]",5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
